# Code for computing predictions for Kaggle

Needs data to be manually downloaded from Kaggle and extracted to be used using `torchvision.datasets.ImageFolder`.

Part of the code comes from https://www.kaggle.com/code/francescolorenzo/96-fine-tuning-resnet34-with-pytorch

In [1]:
import torch

path_to_model = 'epoch_10.pt'

best_model = torch.load(path_to_model, map_location='cpu')

In [5]:
import torchvision
transform = torchvision.transforms.Compose(
        [torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
test_dataset = torchvision.datasets.ImageFolder(root='kaggle_test_data/test/', transform=transform)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=256, shuffle=False, num_workers=2)

In [ ]:
import pandas as pd
import numpy as np

preds = []

best_model.eval()
with torch.no_grad():
    for X, _ in test_dataloader:
        X = X.to('cpu')
        preds.extend(best_model(X).argmax(dim=1).type(torch.int32).cpu().numpy())
        
ids = list(range(1, len(test_dataset)+1))
ids.sort(key=lambda x: str(x))

classes = ('airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
df = pd.DataFrame({'id': ids, 'label': preds})
df['label'] = df['label'].apply(lambda x: classes[x])
df = df.sort_values(by=['id'])
df.to_csv('submission.csv', index=False)